In [1]:
## This contains a simple IK problem for solo 12
## Author : Avadesh Meduri
## Date : 25/02/2021

import time
import crocoddyl
import numpy as np
import pinocchio as pin

from robot_properties_solo.config import Solo12Config
from inverse_kinematics import InverseKinematics

In [2]:
robot = Solo12Config.buildRobotWrapper()
robot_model = robot.model
viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=True)
viz.loadViewerModel()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [170]:
# Running and terminal action models
dt = 1e-3
T = 0.2
N = int(T/dt)

In [171]:
traj = []
com_traj = np.zeros((N,3))
com_traj[:,2] = 0.25
for t in range(N):
    omega = np.pi/N
    x = 0.15*np.sin(2*omega*t)
    com_traj[t][0] = x + 0.2
    traj.append(np.array([0.4 + x , -0.16 ,0.1*np.sin(omega*t)]))
    
traj_hl = traj + np.array([-.4, 0.32, 0])

In [172]:
vel_traj = np.zeros((N, 6))

In [194]:
# setting up running cost model for swing foot task
ik = InverseKinematics(robot_model, dt, T)

# ik.add_com_position_tracking_task(0, T, com_traj, 1e3, "com_task")

ik.add_trajectory_tracking_task(robot_model.getFrameId("FR_FOOT"), 0, T, traj, 1e3, "FR_ftc")
ik.add_trajectory_tracking_task(robot_model.getFrameId("HL_FOOT"), 0, T, traj_hl, 1e3, "HL_ftc")

ik.add_velocity_tracking_cost(robot_model.getFrameId("FL_FOOT"), 0, T, vel_traj, 1e2, "FL_vtc")
# ik.add_velocity_tracking_cost(robot_model.getFrameId("FR_FOOT"), 0, T, vel_traj, 1e2, "FR_vtc")
# ik.add_velocity_tracking_cost(robot_model.getFrameId("HL_FOOT"), 0, T, vel_traj, 1e2, "HL_vtc")
ik.add_velocity_tracking_cost(robot_model.getFrameId("HR_FOOT"), 0, T, vel_traj, 1e2, "HR_vtc")

ik.add_state_regularization_cost(0, T, 1e-4, "xReg")
ik.add_ctrl_regularization_cost(0, T, 1e-7, "uReg")

# setting up terminal cost model
xRegCost = crocoddyl.CostModelState(ik.state)
uRegCost = crocoddyl.CostModelControl(ik.state)
comTrack = crocoddyl.CostModelCoMPosition(ik.state, com_traj[-1], ik.state.nv)

ik.terminalCostModel.addCost("stateReg", xRegCost, 1e-4)
ik.terminalCostModel.addCost("ctrlReg", uRegCost, 1e-7) 
# ik.terminalCostModel.addCost("comtask", comTrack, 1e+5)

In [195]:
ik.setup_costs()

q0 = np.array(Solo12Config.initial_configuration)
x0 = np.concatenate([q0, pin.utils.zero(robot_model.nv)])
xs = ik.optimize(x0) 


In [197]:
for i in range(len(xs)):
    time.sleep(0.01)
    viz.display(xs[i][:robot_model.nq])